# Time to Train my first LLM
---



## Setup

In [20]:
# install unsloth and pandas
#!pip install unsloth
#!pip install pandas

In [21]:
# checking if I can use my gpu
import torch
torch.cuda.is_available()

True

In [22]:
# import the libs that our teacher told us to
from unsloth import FastLanguageModel
import torch

In [23]:
# set the max number of tokens that can be passed in per chunk
max_seq_length = 2048

# let unsloth select our data type for us
dtype = None

# enable 4-bit quantization
# less memory but slightly less accuracy its just some math stuff
load_in_4bit = True

In [24]:
#from google.colab import drive
#drive.mount('/content/drive')

In [25]:
# find path to my data
#!ls 'drive/MyDrive/intro to AI/Notebooks'

In [26]:
import json
import pandas as pd
from datasets import Dataset

# connect to our data path
#dataPath = 'drive/MyDrive/intro to AI/Notebooks/allMethodsCombined.json'

dataPath = '../data/allMethodsCombined.json'

# load and convert to DataFrame
with open(dataPath, 'r') as f:
    data = json.load(f)

df = pd.DataFrame(data)  # <- make it a DataFrame

# save the DataFrame to CSV
df.to_csv('allMethodsCombined.csv', index=False)

# convert to a hf dataset
dataset = Dataset.from_pandas(df)


In [27]:
# make a 2 col dataset
df2col = df[['code', 'javadoc']]

dataset2col = Dataset.from_pandas(df)


In [28]:
df2col.head(4)

,code,javadoc
0,public Point getLoc(){return location;},/*** Gets the location.\n*\n* @return the curr...
1,public void draw(Graphics g){\ng.setColor(colo...,"/*** Draws the ball, and if non-active also dr..."
2,public boolean isDone(){\nreturn finishedInstr...,/*** Checks if the process is completed.\n* @r...
3,public void performInstruction(){\nif (!isDone...,/**Perform a single instruction of the process...


## Model Selections:
- Qwen 2.5 coder (0.5b)
- Codellama (7b)
- Wizardcoder (33b)


In [29]:
# models from hugging face
model1 = 'Qwen/Qwen2.5-Coder-0.5B-Instruct'
model2 = 'WizardLMTeam/WizardCoder-33B-V1.1'
model3 = 'codellama/CodeLlama-7b-hf'


## Fine tuning Qwen 2.5 Coder (0.5B)

### Setting up model

In [30]:
model, tokenizer = FastLanguageModel.from_pretrained(
    # qwen coder 0.5b
    model_name= model1,
    max_seq_length=max_seq_length,
    dtype = dtype,
    # give each model 4bit quantization or however you spell it
    load_in_4bit= True
)

==((====))==  Unsloth 2025.4.3: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 3080 Ti. Num GPUs = 1. Max memory: 12.0 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.7.0+cu118. CUDA: 8.6. CUDA Toolkit: 11.8. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


### Establish How the model will learn

In [31]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj","k_proj","v_proj","o_proj",
                    "gate_proj","up_proj","down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

### Create a prompt for our dataset 

In [32]:
from langchain.prompts import PromptTemplate

oneShotReason = PromptTemplate(
    input_variables=["method_code"],
    template='''\
You are a Java expert. Given a method, write a clear and professional Javadoc comment **first**, followed by a simple, high-school-level explanation of what the method does.


Example:
Method:
public String reverseProcessed(String text) {{
    if (text == null) return null;
    String cleaned = text.trim().toLowerCase();
    return new StringBuilder(cleaned).reverse().toString();
}}

Reasoning:
Checks for null, trims and lowercases the input, then reverses it.

Expected Output:
/**
 * Reverses the characters in the given string after trimming whitespace and converting to lowercase.
 *
 * @param text the input string to process
 * @return the reversed, lowercase, trimmed string
 */

Now analyze and write a Javadoc for the following method:
{method_code}
'''
)


In [33]:
from langchain import PromptTemplate

oneShot = PromptTemplate(
    input_variables=["method_code"],
    template='''\
Write a professional Javadoc comment for the following Java method. Only tell me the javadoc, **do not return any code.** 

Example 1:
Method:
public String reverseProcessed(String text) {{
    if (text == null) return null;
    String cleaned = text.trim().toLowerCase();
    return new StringBuilder(cleaned).reverse().toString();
}}

Javadoc:
/**
 * Reverses the characters in the given string after trimming whitespace and converting to lowercase.
 *
 * @param text the input string to process
 * @return the reversed, lowercase, trimmed string
 */

Example 2:
Method:
private int add(int a, int b) {{
    return a + b;
}}

Javadoc:
/**
 * Adds two integers.
 *
 * @param a the first integer
 * @param b the second integer
 * @return the sum of the two integers
 */

Now write a Javadoc comment for this method:
Method:
{method_code}

Javadoc:
'''
)

In [34]:
# add promt to the dataset
def format_with_prompt(example):
    return {"codeWithPrompt": oneShot.format(method_code=example["code"])}

finalDataset = dataset2col.map(format_with_prompt)


Map:   0%|          | 0/161 [00:00<?, ? examples/s]

In [39]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

finalDataset.select(range(1)).to_pandas()


,javadoc,code,method_name,parameters,return_type,return_variable,called_methods,local_variables,thrown_exceptions,codeWithPrompt
0,/*** Gets the location.\n*\n* @return the current location\n*/,public Point getLoc(){return location;},getLoc,,Point,location,[],[location],[],"Write a professional Javadoc comment for the following Java method. Only tell me the javadoc, **do not return any code.** \n\nExample 1:\nMethod:\npublic String reverseProcessed(String text) {\n if (text == null) return null;\n String cleaned = text.trim().toLowerCase();\n return new StringBuilder(cleaned).reverse().toString();\n}\n\nJavadoc:\n/**\n * Reverses the characters in the given string after trimming whitespace and converting to lowercase.\n *\n * @param text the input string to process\n * @return the reversed, lowercase, trimmed string\n */\n\nExample 2:\nMethod:\nprivate int add(int a, int b) {\n return a + b;\n}\n\nJavadoc:\n/**\n * Adds two integers.\n *\n * @param a the first integer\n * @param b the second integer\n * @return the sum of the two integers\n */\n\nNow write a Javadoc comment for this method:\nMethod:\npublic Point getLoc(){return location;}\n\nJavadoc:\n"


### Start Training (Can be changed)

In [18]:
#add formatting func for some reason
def formatting_func(example):
    return example["codeWithPrompt"]


In [18]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=finalDataset,
    dataset_text_field="codeWithPrompt",
    formatting_func=formatting_func,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        num_train_epochs=2,
        learning_rate=2e-4,
        report_to="none",
    ),
    dataset_num_proc=1,
)



Unsloth: Tokenizing ["codeWithPrompt"]:   0%|          | 0/161 [00:00<?, ? examples/s]

In [19]:
trainer.train()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 161 | Num Epochs = 2 | Total steps = 162
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 1 x 1) = 2
 "-____-"     Trainable parameters = 8,798,208/5,000,000,000 (0.18% trained)


Step,Training Loss


TrainOutput(global_step=162, training_loss=0.002400798378167329, metrics={'train_runtime': 46.2593, 'train_samples_per_second': 6.961, 'train_steps_per_second': 3.502, 'total_flos': 211089470000640.0, 'train_loss': 0.002400798378167329})

## Upload to Hugging Face

In [20]:
from huggingface_hub import login

# Log in first (you only need to do this once)
login()

# Push model
model.push_to_hub("Eyas1/fine_tuned_qwen_java_docs")
tokenizer.push_to_hub("Eyas1/fine_tuned_qwen_java_docs")


  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/35.2M [00:00<?, ?B/s]

Saved model to https://huggingface.co/Eyas1/fine_tuned_qwen_java_docs


README.md:   0%|          | 0.00/628 [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.
